# Scraping: récupération de la page d'accueil du journal Le Soir

Dans ce notebook, nous créons un robot qui va ouvrir la page d'accueil du site du journal [Le Soir](https://www.lesoir.be/) et récupérer le titre de tous les articles du jour et les stocker dans un fichier csv.

## Imports

In [1]:
import re
import time
import requests
from bs4 import BeautifulSoup
import pandas as pd

## Récupération de tous les articles de la page d'accueil



In [2]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}

articles = []

root_url = f"https://www.rtbf.be"
response = requests.get(root_url, headers=headers)
soup = BeautifulSoup(response.text, 'html.parser')
for link in soup.find_all("a", {"class":"stretched-link outline-hidden"}):
    title = link.text.strip()
    url = root_url + link.get('href')
    articles.append([url, title])

In [3]:
# Affichage du nombre d'articles récupérés
len(articles)

80

In [4]:
# Affichage des 10 premières entrées
articles[:10]

[['https://www.rtbf.be/article/france-le-president-emmanuel-macron-dissoudra-l-assemblee-si-le-premier-ministre-sebastien-lecornu-est-censure-11616122',
  "Vers une nouvelle dissolution de l'Assemblée Nationale en France\xa0? Si le gouvernement Lecornu est censuré, Emmanuel Macron renverra aux urnes"],
 ['https://www.rtbf.be/article/european-open-un-tour-et-s-en-va-pour-david-goffin-battu-par-francisco-comesana-en-deux-manches-11616234',
  'European Open - David Goffin fataliste après sa défaite : "Comme un alpiniste qui commencerait à avoir le vertige, je fais n\'importe quoi"'],
 ['https://www.rtbf.be/article/lecornu-suspend-la-reforme-des-retraites-dans-l-espoir-d-eloigner-la-censure-11616196',
  "Lecornu suspend la réforme des retraites jusqu'en 2027, dans l’espoir d’éloigner la censure"],
 ['https://www.rtbf.be/article/the-voice-kids-saison-3-suivez-le-premier-blind-en-direct-11616127',
  'The Voice Kids saison 3 : suivez le premier blind en direct'],
 ['https://www.rtbf.be/articl

## Création d'un dataframe avec les liens et les titres des articles


In [5]:
df = pd.DataFrame(articles, columns=['link', 'title'])
df

,link,title
0,https://www.rtbf.be/article/france-le-presiden...,Vers une nouvelle dissolution de l'Assemblée N...
1,https://www.rtbf.be/article/european-open-un-t...,European Open - David Goffin fataliste après s...
2,https://www.rtbf.be/article/lecornu-suspend-la...,Lecornu suspend la réforme des retraites jusqu...
3,https://www.rtbf.be/article/the-voice-kids-sai...,The Voice Kids saison 3 : suivez le premier bl...
4,https://www.rtbf.be/article/european-open-raph...,European Open : Raphaël Collignon remporte le ...
...,...,...
75,https://www.rtbf.behttps://auvio.rtbf.be/media...,"Venise, les défis hors normes des bâtisseurs"
76,https://www.rtbf.behttps://auvio.rtbf.be/media...,Jardins & Loisirs
77,https://www.rtbf.behttps://auvio.rtbf.be/media...,Plus belle la vie S13
78,https://www.rtbf.behttps://auvio.rtbf.be/media...,Le 13h


In [6]:
# Sauvegarde du dataframe dans un fichier csv
df.to_csv(f"../data/rtbf_{time.strftime('%Y%m%d')}.csv", index=False)

## Téléchargement d'un article et affichage du texte

In [7]:
# Récupération du premier lien du dataframe
article_url = df.iloc[0][0]

# Ouverture de la page
response = requests.get(article_url, headers=headers)
soup = BeautifulSoup(response.content, 'html.parser')
# Parsing du contenu
title = soup.find("h1").get_text(separator=' ')

paragraphs = []
for paragraph in soup.find_all("p", attrs={"class": None}):
    paragraphs.append(paragraph.get_text(separator=' ').strip())

content = "\n".join(paragraphs)
print(title)
print("==================================")
print(content)

Europe
" Suspension immédiate et complète de la réforme des retraites ", " renoncement au 49.3 " pour permettre le vote de mesures de justice fiscale et de pouvoir d'achat, " assouplissement de la trajectoire financière ": le PS a réitéré ses demandes en bureau national, juste avant la déclaration de politique générale du Premier ministre devant l'Assemblée nationale, prévue à15h00.
Alors que les socialistes doivent donc décider dans la foulée de la déclaration de politique générale de Sébastien Lecornu s'ils censurent ou non le gouvernement, le chef de l'État les prévient donc qu'il exclut de nommer un nouveau Premier ministre si M. Lecornu est renversé et qu'il prononcera alors la dissolution de l'Assemblée.
Les désaccords sont estimables, mais ils ne sont acceptables que si des compromis sont possibles
Le président " a constaté que deux motions de censure (de LFI et RN) ont été déposées et que donc certains ne veulent pas débattre. Il a rappelé que les motions de censure qui ont été

### Nettoyage du texte à l'aide d'expressions régulières

In [8]:
# Suppression de tout ce qui se trouve entre parenthèses
clean_content = re.sub("\([^\)]+\)", " ", content)
# Suppression des espaces multiples
clean_content = re.sub("\s+", " ", clean_content)

print(content)
print("==================================")
print(clean_content)


" Suspension immédiate et complète de la réforme des retraites ", " renoncement au 49.3 " pour permettre le vote de mesures de justice fiscale et de pouvoir d'achat, " assouplissement de la trajectoire financière ": le PS a réitéré ses demandes en bureau national, juste avant la déclaration de politique générale du Premier ministre devant l'Assemblée nationale, prévue à15h00.
Alors que les socialistes doivent donc décider dans la foulée de la déclaration de politique générale de Sébastien Lecornu s'ils censurent ou non le gouvernement, le chef de l'État les prévient donc qu'il exclut de nommer un nouveau Premier ministre si M. Lecornu est renversé et qu'il prononcera alors la dissolution de l'Assemblée.
Les désaccords sont estimables, mais ils ne sont acceptables que si des compromis sont possibles
Le président " a constaté que deux motions de censure (de LFI et RN) ont été déposées et que donc certains ne veulent pas débattre. Il a rappelé que les motions de censure qui ont été déposé

### Création d'un fichier avec le contenu de l'article


In [9]:
with open("../data/rtbf_example.txt", "w") as writer:
    writer.write(f"{title}\n\n{clean_content}")

## Pour en savoir plus

- Le web scraping avec Python: https://realpython.com/beautiful-soup-web-scraper-python/
- Tutoriel sur les expressions régulières: https://www.w3schools.com/python/python_regex.asp